## Preliminaries

In [6]:
## lets create a directory to store the images that are uploaded to the server
dir_name = "images_uploaded"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)



## Import Libraries

In [7]:
import io
import uvicorn
import numpy as np
import nest_asyncio
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse

## Deploying The Model Using FastAPI

In [8]:
## lets create an instance of fastapi and call is app
app = FastAPI(title='Deploying a ML model with FastAPI')

## lets list all our models 
class Model(str, Enum):
    yolov3tiny = 'yolov3-tiny'
    yolov3 = 'yolov3'

## lets create an endpoint
@app.get('/')
def home():
    return f'Congratulations! Your API is working as expected. Now head over to http://localhost:8000/docs'

## lets create an edpoint for all the predictions
@app.post('/predict')
def prediction(model: Model, file: UploadFile = File(...)):


    ## 2. lets validate the input file

    filename = file.filename
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not fileExtension:
        raise HTTPException(status_code=415, detail='Unsupported file provided.')

    ## 2. lets transform the raw image into a cv2 file

    ## lets read image as a stream of bytes
    image_stream = io.BytesIO(file.file.read())

    ## lets start the stream from the beginning
    image_stream.seek(0)

    ## lets write the stream of bytes into a numpy array
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)

    ## lets decode the numpy array as an image
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR) 

    ## 3. lets run the object detection model

    ## lets run object detection
    bbox, label, conf = cv.detect_common_objects(image, model=model)

    ## lets create image that includes bounding boxes and labels
    output_image = draw_bbox(image, bbox, label, conf)

    ## lets save it in a folder within the server
    cv2.imwrite(f'images_uploaded/{filename}', output_image)

    ## 4. lets stream the response back to the client

    ## open the saved image for reading in binary model
    file_image = open(f'images_uploaded/{filename}', mode='rb')

    ## lets return the image as a stream specifying media type
    return StreamingResponse(file_image, media_type="image/jpeg")


In [9]:
## lets allow the server to run interactively
nest_asyncio.apply()
 
host = '0.0.0.0' if os.getenv('DOCKER-SETUP') else '127.0.0.1'
##
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [14985]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55358 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:55358 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:35692 - "POST /predict?model=yolov3-tiny HTTP/1.1" 200 OK
INFO:     127.0.0.1:46036 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14985]
